Import all necessary libraries and databases:

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_json('unzipped_path/data.json')
df

Set variables:

In [ ]:
VAR_calibration = 737
VAR_threshold = 1000

In [ ]:
#def remove_outliers():
    
        

Function to remove centre beam:

In [ ]:
def remove_centre_beam(xray_image):
    threshold_value = VAR_threshold
    primary_beam_mask = xray_image > threshold_value
    true_indices = np.argwhere(primary_beam_mask)
    min_row = true_indices[:, 0].min()
    max_row = true_indices[:, 0].max()
    min_col = true_indices[:, 1].min()
    max_col = true_indices[:, 1].max()
    isolated_beam = np.zeros_like(xray_image)
    isolated_beam[min_row:max_row+1, min_col:max_col+1] = xray_image[min_row:max_row+1, min_col:max_col+1]
    return isolated_beam

In [ ]:
def detect_lb(xray_image):
    threshold_value = VAR_threshold
    primary_beam_mask = xray_image > threshold_value
    true_indices = np.argwhere(primary_beam_mask)
    min_row = true_indices[:, 0].min()
    max_row = true_indices[:, 0].max()
    min_col = true_indices[:, 1].min()
    max_col = true_indices[:, 1].max()
    #if (max_col<128):
        #left
    #else:
        #right
    isolated_beam = np.zeros_like(xray_image)
    isolated_beam[min_row:max_row+1, min_col:max_col+1] = xray_image[min_row:max_row+1, min_col:max_col+1]
    return isolated_beam


In [ ]:
def reflect_left_to_bottom(image):
    transposed = np.transpose(image)
    transposed = transposed[::-1]
    return transposed

In [ ]:
def reflect_bottom_to_left(image):
    transposed = np.transpose(image)
    transposed = transposed[:, ::-1]
    return transposed

In [ ]:
from collections import Counter
def check_out_738(num):
    outliers = [278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309]
    if outliers.count(num)>0:
        return False
    return True

In [ ]:
from collections import Counter
def check_out_737(num):
    outliers = [267,268,269,270,271,272,273,274,275,290,291,292,293,294,295,296,297,298,186]
    if outliers.count(num)>0:
        return False
    return True

Create an averaged matrix of all samples within the calibration:

In [ ]:
totals= np.zeros([256, 256]) 
number_samples=0
outlier_counter=0
for index,row in df.iterrows():
    if row['calibration_measurement_id']==VAR_calibration :
        ###for 737
        if check_out_737(outlier_counter):
            xray_image = np.array(row['measurement_data'])
            isolated_beam = remove_centre_beam(xray_image)
            xray_image = np.fft.fft2(xray_image)
            isolated_beam = np.fft.fft2(isolated_beam)
            img = np.absolute(np.subtract(xray_image,isolated_beam)) ## full pic in fourier space
            img = np.divide(img, img[0][0]) ## normalisation
            totals = np.add(totals,img)  ## create an averaged matrix of all samples
            number_samples+=1
        outlier_counter+=1
totals = np.divide(totals, number_samples)

Create a flattened matrix of all standardised samples:

In [ ]:
matrix = np.empty([number_samples, 65536])
counter = 0
outlier_counter=0
for index,row in df.iterrows():
    if row['calibration_measurement_id']==VAR_calibration:
        if check_out_737(outlier_counter):
            xray_image = np.array(row['measurement_data'])
            isolated_beam = remove_centre_beam(xray_image)
            xray_image = np.fft.fft2(xray_image)
            isolated_beam = np.fft.fft2(isolated_beam)
            img = np.absolute(np.subtract(xray_image,isolated_beam)) ## full pic in fourier space
            img = np.divide(img, img[0][0]) ## normalisation
            img = np.divide(img,totals) ## divide by average - standartisation
            img = np.ravel(img) ## flatten
            matrix[counter] = img
            counter+=1
        outlier_counter+=1

Create a "key" for all t/f values of the samples:

In [ ]:
key = []
outlier_counter=0
for index,row in df.iterrows():
    if row['calibration_measurement_id']==VAR_calibration:
        if check_out_737(outlier_counter):
            key+= [row['cancer_tissue']]
        outlier_counter+=1

Perform PCA:

In [ ]:
pca = PCA(n_components=3)
fitted = pca.fit(matrix)
transformed = pca.transform(matrix)
for line in transformed:
    print(line)

Graph PCA:

In [ ]:
x = transformed[:, 0]
y = transformed[:, 1]
z = transformed[:, 2]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

counter = 0
for item in key:
    if item:
        ax.scatter(transformed[counter, 0], transformed[counter, 1], transformed[counter, 2], c='blue', label='Cancer')
    else:
        ax.scatter(transformed[counter, 0], transformed[counter, 1], transformed[counter, 2], c='red', label='Non-Cancer') 
    counter+=1

ax.set_xlabel('pc0')
ax.set_ylabel('pc1')
ax.set_zlabel('pc2')
plt.title('calibration '+str(VAR_calibration))
#plt.show()

In [ ]:
for index in range(len(x)):
    if transformed[index][0]>200.0:
        print(index)
        print (transformed[index])